# Training classifiers on HOG Data
#### For extraction of features, see extractFeatures.ipynb

conda env- make sure to switch to python3 kernel, faus_dl

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import sklearn
from sklearn import datasets, svm, metrics
import bcolz
import os, sys
import datetime as dt
import random

In [2]:
import pickle

In [3]:
DISFA_path='/media/amogh/Stuff/CMU/datasets/DISFA_data/'
DISFA_AU_path=DISFA_path+'ActionUnit_Labels/'
print(DISFA_AU_path)
Videos_right_path=DISFA_path+'Videos_RightCamera/'
print(Videos_right_path)

/media/amogh/Stuff/CMU/datasets/DISFA_data/ActionUnit_Labels/
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/


Helper Functions

## Training Details explained

Loading the saved features:

In [5]:
dir_features_hog_1_fau4_1=DISFA_path+"/features/hog_1/fau4_1/"
dir_features_hog_1_fau4_1

'/media/amogh/Stuff/CMU/datasets/DISFA_data//features/hog_1/fau4_1/'

In [6]:
features_fau4_1_hog_1_positives=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'positives/',mode='r')
print("features_fau4_1_hog_1_positives loaded, shape is: ",features_fau4_1_hog_1_positives.shape)

features_fau4_1_hog_1_positives loaded, shape is:  (2000, 448)


In [7]:
features_fau4_1_hog_1_negatives=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'negatives/',mode='r')
print("features_fau4_1_hog_1_negatives loaded, shape is : ",features_fau4_1_hog_1_negatives.shape)

features_fau4_1_hog_1_negatives loaded, shape is :  (1898, 448)


Loading test_features:

In [8]:
features_fau4_1_hog_1_positives_test=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'positives_test/',mode='r')
print("features_fau4_1_hog_1_positives_test loaded, shape is: ",features_fau4_1_hog_1_positives_test.shape)

features_fau4_1_hog_1_positives_test loaded, shape is:  (464, 448)


In [9]:
features_fau4_1_hog_1_negatives_test=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'negatives_test/',mode='r')
print("features_fau4_1_hog_1_negatives_test loaded, shape is : ",features_fau4_1_hog_1_negatives_test.shape)

features_fau4_1_hog_1_negatives_test loaded, shape is :  (499, 448)


Defining the training and testing data

In [10]:
train_no=1800
X=np.concatenate([features_fau4_1_hog_1_positives[:train_no],features_fau4_1_hog_1_negatives[:train_no]])
X.shape

(3600, 448)

In [11]:
Y=np.concatenate([np.repeat(1,train_no),np.repeat(0,train_no)])
Y.shape

(3600,)

In [12]:
X_train,X_test,y_train,y_test=sklearn.model_selection.train_test_split(X,Y,test_size=0.15,random_state=42)

Defining the parameter range for SVM:

In [13]:
gamma_range=np.outer(np.logspace(-3,0,4),np.array([1,5]))
gamma_range=gamma_range.flatten()
C_range=np.outer(np.logspace(-1,1,3),np.array([1,5]))
C_range=C_range.flatten()
parameters={'kernel':['rbf'],'C':C_range,'gamma':gamma_range}

In [14]:
svm_clsf=svm.SVC()
grid_clsf=sklearn.model_selection.GridSearchCV(estimator=svm_clsf,param_grid=parameters,n_jobs=8,verbose=2)

Start training and parameter search 

In [15]:
start_time=dt.datetime.now()
print('Start param searching at {}'.format(str(start_time)))
grid_clsf.fit(X_train,y_train)
elapsed_time=dt.datetime.now()-start_time
print('Elapsed time, param searching {}'.format(str(elapsed_time)))
sorted(grid_clsf.cv_results_.keys())

Start param searching at 2018-06-07 10:23:59.808611
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=  11.2s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=  12.3s
[CV] C=0.1, gamma=0.05, kernel=rbf ...................................
[CV] ................... C=0.1, ga

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  1.2min


[CV] ................... C=0.5, gamma=0.001, kernel=rbf, total=  13.0s
[CV] C=0.5, gamma=0.05, kernel=rbf ...................................
[CV] ................... C=0.5, gamma=0.005, kernel=rbf, total=  11.1s
[CV] C=0.5, gamma=0.05, kernel=rbf ...................................
[CV] ................... C=0.5, gamma=0.005, kernel=rbf, total=  12.5s
[CV] C=0.5, gamma=0.05, kernel=rbf ...................................
[CV] .................... C=0.5, gamma=0.01, kernel=rbf, total=  10.3s
[CV] C=0.5, gamma=0.1, kernel=rbf ....................................
[CV] ................... C=0.5, gamma=0.005, kernel=rbf, total=  11.7s
[CV] C=0.5, gamma=0.1, kernel=rbf ....................................
[CV] .................... C=0.5, gamma=0.01, kernel=rbf, total=  10.7s
[CV] C=0.5, gamma=0.1, kernel=rbf ....................................
[CV] ................... C=0.5, gamma=0.001, kernel=rbf, total=  14.4s
[CV] C=0.5, gamma=0.5, kernel=rbf ....................................
[CV] .

[Parallel(n_jobs=8)]: Done 144 out of 144 | elapsed:  4.5min finished


Elapsed time, param searching 0:04:36.259007


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_C',
 'param_gamma',
 'param_kernel',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [16]:
classifier=grid_clsf.best_estimator_
params=grid_clsf.best_params_

scores=grid_clsf.cv_results_['mean_test_score'].reshape(len(C_range),len(gamma_range))
# plot_param_space_scores(scores,C_range, gamma_range)

In [17]:
scores

array([[0.50915033, 0.60130719, 0.78398693, 0.81143791, 0.83529412,
        0.83267974, 0.8003268 , 0.50915033],
       [0.60163399, 0.75980392, 0.81699346, 0.85784314, 0.88660131,
        0.92614379, 0.91862745, 0.50915033],
       [0.78039216, 0.81601307, 0.84019608, 0.89509804, 0.92287582,
        0.94411765, 0.93888889, 0.61437908],
       [0.81699346, 0.86699346, 0.89509804, 0.94248366, 0.95326797,
        0.95490196, 0.95163399, 0.64379085],
       [0.84215686, 0.89542484, 0.91699346, 0.9503268 , 0.95196078,
        0.95490196, 0.95163399, 0.64379085],
       [0.89705882, 0.92745098, 0.93529412, 0.94542484, 0.94803922,
        0.95490196, 0.95163399, 0.64379085]])

In [18]:
expected = y_test
predicted = classifier.predict(X_test)
print(predicted)
print("predicted and expected are same in: ", np.sum(expected==predicted)," out of ", expected.shape[0])

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
      
cm = metrics.confusion_matrix(expected, predicted)
print("Confusion matrix:\n%s" % cm)

# plot_confusion_matrix(cm)

print("Accuracy={}".format(metrics.accuracy_score(expected, predicted)))


[1 1 0 0 0 1 1 1 0 1 0 1 1 1 1 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1
 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 1 0
 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1
 0 1 0 0 1 0 1 0 1 0 0 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 1
 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1
 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 0
 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1
 1 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 1 0
 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 0 0 0
 0 0 0 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0
 1 1 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 1 1 1 0 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 1 0 0 0 0 1 0 

In [19]:
print(classifier.predict(features_fau4_1_hog_1_positives_test).sum()," out of ", len(features_fau4_1_hog_1_positives_test))

14  out of  464


In [20]:
print(classifier.predict(features_fau4_1_hog_1_negatives_test).sum()," out of ", len(features_fau4_1_hog_1_positives_test))

0  out of  464


**Saving the model to disk**

In [32]:
DISFA_path

'/media/amogh/Stuff/CMU/datasets/DISFA_data/'

In [33]:
models_path=DISFA_path+'models/'
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [34]:
with open(models_path+'fau4_1_hog_1.pickle','wb') as handle:
    pickle.dump(classifier,handle)

**Loading a model from disk**

In [35]:
with open(models_path+'fau4_1_hog_1.pickle','rb') as handle:
    clf1=pickle.load(handle)

**Calculating features for a cropped image from disk**

In [23]:
dir_cropped_FAU4_1=DISFA_path+"/features/cropped_images/FAU4_1/"
print(dir_cropped_FAU4_1)
dir_cropped_FAU4_1_positives=dir_cropped_FAU4_1+"positives/"
print(dir_cropped_FAU4_1_positives)
dir_cropped_FAU4_1_negatives=dir_cropped_FAU4_1+"negatives/"
print(dir_cropped_FAU4_1_negatives)

/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/
/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/positives/
/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/negatives/


In [24]:
list_cropped_fau4_1_positive_files=os.listdir(dir_cropped_FAU4_1_positives)
list_cropped_fau4_1_positive_files[0]

'RightVideoSN001_1340.png'

In [44]:
classifier.predict(features_fau4_1_hog_1_positives)

array([1, 1, 1, ..., 1, 1, 1])

# Old Trained Model (before June 8)

In [5]:
dir_features_hog_2_fau4_1=DISFA_path+"/features/hog_2/fau4_1/"
dir_features_hog_2_fau4_1

'/media/amogh/Stuff/CMU/datasets/DISFA_data//features/hog_2/fau4_1/'

Training or FAU 4 HOG2(Affectiva configuration)

**RBF SVM training**

In [ ]:
X,Y=loadTrainData(dir_features_hog_2_fau4_1)
best_classifier,param,scores=trainModel(X,Y)

In [21]:
features_folder=dir_features_hog_2_fau4_1
features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
print("features_positives_test loaded, shape is: ",features_positives_test.shape)
features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)

features_positives_test loaded, shape is:  (469, 1056)
features_negatives_test loaded, shape is:  (498, 1056)


In [26]:
print (best_classifier.predict(features_positives_test).sum(), " out of ", len(features_positives_test),best_classifier.predict(features_positives_test).sum()/len(features_positives_test) )

294  out of  469 0.6268656716417911


In [29]:
print (best_classifier.predict(features_negatives_test).sum(), " out of ", len(features_negatives_test),(len(features_negatives_test)-best_classifier.predict(features_negatives_test).sum())/len(features_negatives_test))

108  out of  498 0.7831325301204819


In [35]:
DISFA_path

'/media/amogh/Stuff/CMU/datasets/DISFA_data/'

In [36]:
models_path=DISFA_path+'models/'
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [38]:
with open(models_path+'fau4_1_hog_2_rbf.pickle','wb') as handle:
    pickle.dump(best_classifier,handle)

**Linear SVM training**

In [45]:
def trainModel(X_train,y_train):
    #setup
    gamma_range=np.outer(np.logspace(-3,0,4),np.array([1,5]))
    gamma_range=gamma_range.flatten()
    C_range=np.outer(np.logspace(-1,1,3),np.array([1,5]))
    C_range=C_range.flatten()
    parameters={'kernel':['linear'],'C':C_range,'gamma':gamma_range}
    svm_clsf=svm.SVC()
    grid_clsf=sklearn.model_selection.GridSearchCV(estimator=svm_clsf,param_grid=parameters,n_jobs=1,verbose=2)
    #train
    start_time=dt.datetime.now()
    print('Start param searching at {}'.format(str(start_time)))
    grid_clsf.fit(X_train,y_train)
    elapsed_time=dt.datetime.now()-start_time
    print('Elapsed time, param searching {}'.format(str(elapsed_time)))
    sorted(grid_clsf.cv_results_.keys())
    #choose classifier
    classifier=grid_clsf.best_estimator_
    params=grid_clsf.best_params_
    scores=grid_clsf.cv_results_['mean_test_score'].reshape(2,len(C_range),len(gamma_range))
    print("scores are: ", scores)
    return classifier, params,scores

In [31]:
X,Y=loadTrainData(dir_features_hog_2_fau4_1)
best_classifier,param,scores=trainModel(X,Y)

features_positives loaded, shape is:  (1999, 1056)
features_negatives loaded, shape is :  (2000, 1056)
training data shape is:  (3998, 1056)
labels shape is:  (3998,)
Start param searching at 2018-06-06 16:47:41.880015
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  10.0s
[CV] C=0.1, gamma=0.001, kernel=linear ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.4s remaining:    0.0s


[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=   9.5s
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  10.0s
[CV] C=0.1, gamma=0.005, kernel=linear ...............................
[CV] ................ C=0.1, gamma=0.005, kernel=linear, total=   9.8s
[CV] C=0.1, gamma=0.005, kernel=linear ...............................
[CV] ................ C=0.1, gamma=0.005, kernel=linear, total=  10.2s
[CV] C=0.1, gamma=0.005, kernel=linear ...............................
[CV] ................ C=0.1, gamma=0.005, kernel=linear, total=  10.2s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] ................. C=0.1, gamma=0.01, kernel=linear, total=  10.0s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] ................. C=0.1, gamma=0.01, kernel=linear, total=   9.9s
[CV] C=0.1, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 24.1min finished


Elapsed time, param searching 0:24:08.732567
scores are:  [[0.74462231 0.74462231 0.74462231 0.74462231 0.74462231 0.74462231
  0.74462231 0.74462231]
 [0.80490245 0.80490245 0.80490245 0.80490245 0.80490245 0.80490245
  0.80490245 0.80490245]
 [0.84242121 0.84242121 0.84242121 0.84242121 0.84242121 0.84242121
  0.84242121 0.84242121]
 [0.87818909 0.87818909 0.87818909 0.87818909 0.87818909 0.87818909
  0.87818909 0.87818909]
 [0.89294647 0.89294647 0.89294647 0.89294647 0.89294647 0.89294647
  0.89294647 0.89294647]
 [0.8991996  0.8991996  0.8991996  0.8991996  0.8991996  0.8991996
  0.8991996  0.8991996 ]]


In [32]:
features_folder=dir_features_hog_2_fau4_1
features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
print("features_positives_test loaded, shape is: ",features_positives_test.shape)
features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)

features_positives_test loaded, shape is:  (469, 1056)
features_negatives_test loaded, shape is:  (498, 1056)


In [44]:
print (best_classifier.predict(features_positives_test).sum(), " out of ", len(features_positives_test),best_classifier.predict(features_positives_test).sum()/len(features_positives_test) )

407  out of  469 0.8678038379530917


In [43]:
print (best_classifier.predict(features_negatives_test).sum(), " out of ", len(features_negatives_test),(len(features_negatives_test)-best_classifier.predict(features_negatives_test).sum())/len(features_negatives_test))

154  out of  498 0.6907630522088354


In [39]:
DISFA_path

'/media/amogh/Stuff/CMU/datasets/DISFA_data/'

In [40]:
models_path=DISFA_path+'models/'
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [42]:
with open(models_path+'fau4_1_hog_2_linear.pickle','wb') as handle:
    pickle.dump(best_classifier,handle)

## Automating loading features, training process 

In [38]:
def loadTrainData(features_folder,train_no,boolShuffle=True):
    #loading features
    features_positives=bcolz.carray(rootdir=features_folder+'positives/',mode='r')
    print("features_positives loaded, shape is: ",features_positives.shape)
    features_negatives=bcolz.carray(rootdir=features_folder+'negatives/',mode='r')
    print("features_negatives loaded, shape is : ",features_negatives.shape)
    
#     features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
#     print("features_positives_test loaded, shape is: ",features_positives_test.shape)
#     features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
#     print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)
    
    #setting up the training data
    train_no=np.min([features_positives.shape[0],features_negatives.shape[0],train_no])
#     if boolShuffle:
#         random.shuffle(features_positives)
#         random.shuffle(features_negatives)
    X=np.concatenate([features_positives[:train_no],features_negatives[:train_no]])
    print("training data shape is: ",X.shape)
    #setting labels/targets
    Y=np.concatenate([np.repeat(1,train_no),np.repeat(0,train_no)])
    print("labels shape is: ",Y.shape)
    return X, Y

In [46]:
def trainModel(X_train,y_train,no_jobs):
    #setup
    gamma_range=np.outer(np.logspace(-3,0,4),np.array([1,5]))
    gamma_range=gamma_range.flatten()
    C_range=np.outer(np.logspace(-1,1,3),np.array([1,5]))
    C_range=C_range.flatten()
    parameters={'kernel':['rbf','linear'],'C':C_range,'gamma':gamma_range}
    svm_clsf=svm.SVC()
    grid_clsf=sklearn.model_selection.GridSearchCV(estimator=svm_clsf,param_grid=parameters,n_jobs=no_jobs,verbose=2)
    #train
    start_time=dt.datetime.now()
    print('Start param searching at {}'.format(str(start_time)))
    grid_clsf.fit(X_train,y_train)
    elapsed_time=dt.datetime.now()-start_time
    print('Elapsed time, param searching {}'.format(str(elapsed_time)))
    sorted(grid_clsf.cv_results_.keys())
    #choose classifier
    classifier=grid_clsf.best_estimator_
    params=grid_clsf.best_params_
    scores=grid_clsf.cv_results_['mean_test_score'].reshape(2,len(C_range),len(gamma_range))
    print("scores are: ", scores)
    return classifier, params,scores

In [47]:
def trainFinal(features_folder,no_jobs,train_no=2000):
    X,Y=loadTrainData(features_folder,train_no,True)
    classifier,params,scores=trainModel(X,Y,no_jobs)
    print(scores)
    return classifier, params,scores

In [58]:
def test(features_folder):
    features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
    print("features_positives_test loaded, shape is: ",features_positives_test.shape)
    features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
    print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)
    print (classifier_fau2.predict(features_positives_test).sum(), " positives out of ", len(features_positives_test),classifier_fau2.predict(features_positives_test).sum()/len(features_positives_test) )
    print (classifier_fau2.predict(features_negatives_test).sum(), " positives out of ", len(features_negatives_test),(len(features_negatives_test)-classifier_fau2.predict(features_negatives_test).sum())/len(features_negatives_test))

In [ ]:
FAU2_1

In [49]:
hog_fau2_1_folder=DISFA_path+'/features/hog_FAU2_1_o_6_ppc_8_cpb_4/'

classifier_fau2,params_fau2,scores_fau2=trainFinal(hog_fau2_1_folder,8,2000)

features_positives loaded, shape is:  (2000, 1056)
features_negatives loaded, shape is :  (1999, 1056)
training data shape is:  (3998, 1056)
labels shape is:  (3998,)
Start param searching at 2018-06-09 00:25:54.905122
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................ C=0.1, gamma=0.001, kernel=linear, total=  25.1s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ....

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  3.4min


[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  41.3s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  25.6s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  28.2s
[CV] C=0.1, gamma=0.5, kernel=linear .................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  26.8s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] .................. C=0.1, gamma=0.1, kernel=linear, total=  27.4s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.5, kernel=rbf, total=  26.2s
[CV] C=0.1, gamma=1.0, kernel=rbf ....................................
[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total=  40.4s
[CV] C=0.1, gamma=1.0, kernel=linear .................................
[CV] .

[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed: 13.6min


[CV] ................ C=5.0, gamma=0.001, kernel=linear, total=  17.5s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  16.3s
[CV] C=5.0, gamma=0.005, kernel=linear ...............................
[CV] ................ C=5.0, gamma=0.005, kernel=linear, total=  15.7s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  41.2s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  40.9s
[CV] C=5.0, gamma=0.01, kernel=rbf ...................................
[CV] ................... C=5.0, gamma=0.005, kernel=rbf, total=  34.3s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] ................... C=5.0, gamma=0.001, kernel=rbf, total=  43.8s
[CV] C=5.0, gamma=0.01, kernel=linear ................................
[CV] .

[Parallel(n_jobs=8)]: Done 288 out of 288 | elapsed: 22.1min finished


Elapsed time, param searching 0:22:10.577138
scores are:  [[[0.75537769 0.8004002  0.75537769 0.8004002  0.75487744 0.8004002
   0.75512756 0.8004002 ]
  [0.75637819 0.8004002  0.7993997  0.8004002  0.80315158 0.8004002
   0.79989995 0.8004002 ]
  [0.75537769 0.80490245 0.75537769 0.80490245 0.75487744 0.80490245
   0.79564782 0.80490245]
  [0.8004002  0.80490245 0.80465233 0.80490245 0.80390195 0.80490245
   0.82491246 0.80490245]
  [0.75537769 0.80215108 0.75537769 0.80215108 0.75712856 0.80215108
   0.8004002  0.80215108]
  [0.80390195 0.80215108 0.8024012  0.80215108 0.80365183 0.80215108
   0.83491746 0.80215108]]

 [[0.75537769 0.81490745 0.79564782 0.81490745 0.8004002  0.81490745
   0.80515258 0.81490745]
  [0.80190095 0.81490745 0.82041021 0.81490745 0.83091546 0.81490745
   0.85642821 0.81490745]
  [0.75787894 0.82841421 0.8004002  0.82841421 0.80415208 0.82841421
   0.80165083 0.82841421]
  [0.80015008 0.82841421 0.83291646 0.82841421 0.84017009 0.82841421
   0.86543272 0.82

In [54]:
features_folder=hog_fau2_1_folder
features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
print("features_positives_test loaded, shape is: ",features_positives_test.shape)
features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)

features_positives_test loaded, shape is:  (500, 1056)
features_negatives_test loaded, shape is:  (498, 1056)


In [55]:
#testing

print (classifier_fau2.predict(features_positives_test).sum(), " out of ", len(features_positives_test),classifier_fau2.predict(features_positives_test).sum()/len(features_positives_test) )

print (classifier_fau2.predict(features_negatives_test).sum(), " positives out of ", len(features_negatives_test),(len(features_negatives_test)-classifier_fau2.predict(features_negatives_test).sum())/len(features_negatives_test))

In [53]:
classifier_fau2.predict()

TypeError: predict() missing 1 required positional argument: 'X'

In [50]:
hog_fau1_1_folder=DISFA_path+'/features/hog_FAU1_1_o_6_ppc_8_cpb_4/'

classifier_fau1,params_fau1,scores_fau1=trainFinal(hog_fau1_1_folder,8,2000)

features_positives loaded, shape is:  (1994, 1056)
features_negatives loaded, shape is :  (0, 1056)
training data shape is:  (0, 1056)
labels shape is:  (0,)
Start param searching at 2018-06-09 00:48:05.558790
Fitting 3 folds for each of 96 candidates, totalling 288 fits


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.